In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
def extract_lhg(filename: str):

    pattern = r"L(\d+)H(\d+)G(\d+)"
    match = re.search(pattern, filename)

    if not match:
        return None
    
    L, H, G = map(int, match.groups())
    return {"L": L, "H": H, "G": G}

In [ ]:
def despike_signal(signal: np.ndarray):
    """
    Aplica los primeros pasos del método Phase-Space Thresholding:
    1) Cálculo de derivadas Du y D2u (sin dividir por dt)
    2) Cálculo de desviaciones estándar y Universal Threshold

    Por ahora NO modifica la señal: clean_signal = signal original.
    Más adelante, usaremos Du, D2u, sigmas y lambda_U para detectar y reemplazar spikes.
    """
    # Aseguramos tipo numpy array float
    signal = np.asarray(signal, dtype=float)

    # --- Paso 0 (recomendación del paper): quitar la media ---
    # "Before applying any of these methods, we remove the mean ..." (paper, sección Algorithms)
    mean_u = np.nanmean(signal)
    u = signal - mean_u  # trabajamos con señal centrada en 0

    n = u.size

    # --- Paso 1: derivadas centradas Du y D2u (sin dividir por Δt) ---
    # Ecs. (7) y (8) del paper:
    #   Du_i  = (u_{i+1} - u_{i-1}) / 2
    #   D2u_i = (Du_{i+1} - Du_{i-1}) / 2
    # Nota: no dividimos por dt, tal como indica el paper.

    Du = np.empty_like(u)
    D2u = np.empty_like(u)

    # Para los puntos interiores usamos diferencias centradas
    Du[1:-1] = (u[2:] - u[:-2]) / 2.0
    # Bordes: usamos una aproximación simple para no dejarlos indefinidos
    Du[0] = Du[1]
    Du[-1] = Du[-2]

    # Segunda "derivada" a partir de Du
    D2u[1:-1] = (Du[2:] - Du[:-2]) / 2.0
    D2u[0] = D2u[1]
    D2u[-1] = D2u[-2]

    # --- Paso 2: desviaciones estándar y Universal Threshold ---
    # su, sDu, sD2u y luego λ_U = sqrt(2 ln n) (Ecs. (1)-(2) del paper).
    su = np.nanstd(u, ddof=0)
    sDu = np.nanstd(Du, ddof=0)
    sD2u = np.nanstd(D2u, ddof=0)

    lambda_U = np.sqrt(2.0 * np.log(n)) # λ_U = sqrt(2 ln n)

    # ---------- Paso 3: ángulo de rotación θ entre u y D2u ----------
    # Eq. (9): θ = atan( sum(u_i * D2u_i) / sum(u_i^2) )
    # Usamos arctan2 por robustez numérica.
    sum_u_D2u = np.nansum(u * D2u)
    sum_u2 = np.nansum(u * u)

    if sum_u2 == 0.0:
        # Caso degenerado: toda la señal es casi constante
        theta = 0.0
    else:
        theta = np.arctan2(sum_u_D2u, sum_u2)
    

    # ---------- Paso 4: parámetros de las elipses en las 3 proyecciones ----------

    # 4.A) Elipse en (u, Du)
    #   eje mayor  = λ_U * su  (dirección u)
    #   eje menor  = λ_U * sDu (dirección Du)
    a_u_Du = lambda_U * su
    b_u_Du = lambda_U * sDu

    # 4.B) Elipse en (Du, D2u)
    #   eje mayor  = λ_U * sDu   (dirección Du)
    #   eje menor  = λ_U * sD2u  (dirección D2u)
    a_Du_D2u = lambda_U * sDu
    b_Du_D2u = lambda_U * sD2u

    # 4.C) Elipse en (u, D2u) rotada un ángulo θ
    # Queremos encontrar a y b tales que:
    #   (λ_U su)^2     = a^2 cos^2 θ + b^2 sin^2 θ
    #   (λ_U sD2u)^2   = a^2 sin^2 θ + b^2 cos^2 θ
    A = (lambda_U * su) ** 2
    B = (lambda_U * sD2u) ** 2

    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    cos2 = cos_t * cos_t
    sin2 = sin_t * sin_t

    # Determinante del sistema (equivalente a cos(2θ))
    denom = cos2 * cos2 - sin2 * sin2  # = cos^4 θ - sin^4 θ = cos(2θ)(cos^2+sin^2)=cos(2θ)

    if np.isclose(denom, 0.0):
        # Sistema casi degenerado: tomamos sin rotación explícita
        a_u_D2u = np.sqrt(max(A, 0.0))
        b_u_D2u = np.sqrt(max(B, 0.0))
    else:
        # Resolver para a^2 y b^2
        a2 = (A * cos2 - B * sin2) / denom
        b2 = (B * cos2 - A * sin2) / denom

        # Aseguramos no negativos (por redondeos numéricos)
        a2 = max(a2, 0.0)
        b2 = max(b2, 0.0)

        a_u_D2u = np.sqrt(a2)
        b_u_D2u = np.sqrt(b2)
    
    # ---------- Paso 5: detección de puntos dentro/fuera de elipse ----------

    # Inicialmente asumimos que TODO es "bueno"
    inside_all = np.ones_like(u, dtype=bool)

    # 5.A) Proyección (u, Du) → elipse centrada en (0,0) con semiejes (a_u_Du, b_u_Du)
    if a_u_Du > 0 and b_u_Du > 0:
        x = u
        y = Du
        # Ecuación de elipse: (x/a)^2 + (y/b)^2 <= 1 → dentro
        inside_u_Du = (x / a_u_Du) ** 2 + (y / b_u_Du) ** 2 <= 1.0
        inside_all &= inside_u_Du  # debe ser True en todas las proyecciones
    # Si alguno de los ejes = 0, simplemente no usamos esta proyección.

    # 5.B) Proyección (Du, D2u)
    if a_Du_D2u > 0 and b_Du_D2u > 0:
        x = Du
        y = D2u
        inside_Du_D2u = (x / a_Du_D2u) ** 2 + (y / b_Du_D2u) ** 2 <= 1.0
        inside_all &= inside_Du_D2u

    # 5.C) Proyección rotada (u, D2u)
    if a_u_D2u > 0 and b_u_D2u > 0:
        # Rotamos cada punto al sistema de ejes principales de la elipse
        # x' =  u cosθ + D2u sinθ
        # y' = -u sinθ + D2u cosθ
        x_prime = u * cos_t + D2u * sin_t
        y_prime = -u * sin_t + D2u * cos_t

        inside_u_D2u = (x_prime / a_u_D2u) ** 2 + (y_prime / b_u_D2u) ** 2 <= 1.0
        inside_all &= inside_u_D2u

    # Todo lo que NO está dentro de las 3 elipses → spike
    spike_mask = ~inside_all


    clean_signal = signal.copy()        # luego acá pondremos la señal corregida

    

    return clean_signal, spike_mask


In [4]:
VELOCITY_COLUMNS = [
    "V1/X/E(cm/s)",
    "V2/Y/N(cm/s)",
    "V3/Z/U(cm/s)",
]

def process_csv_file(filename: str, output_suffix: str = "_clean"):

    print(f"Procesando archivo: {filename}")
    
    data_info = extract_lhg(filename)
    print("Info LHG:", data_info)

    file_path = './raw_data/' + filename
    
    df = pd.read_csv(file_path, sep="\t")
    
    for col in VELOCITY_COLUMNS:

        if col not in df.columns:
            print(f"Algo mal. columna {col} no encontrada en {filename}, se omite.")
            continue
        
        print(f"  → Despiking en columna: {col}")
        signal = df[col].values
        
        clean_signal, spike_mask = despike_signal(signal)
        
        clean_col = col + "_clean"
        mask_col = col + "_is_spike"
        
        df[clean_col] = clean_signal
        df[mask_col] = spike_mask
    
    base, ext = os.path.splitext(filename)
    output_name = f"./clean_data/{base}{output_suffix}{ext}"
    
    df.to_csv(output_name, index=False, sep="\t")
    print(f"  ✅ Guardado como: {output_name}\n")


In [5]:
csv_files = [file for file in os.listdir(os.getcwd() + '/raw_data') if file.endswith(".csv")]

print(f"Se encontraron {len(csv_files)} archivos CSV.\n")

for i, filename in enumerate(csv_files, start=1):
    print(f"[{i}/{len(csv_files)}]")
    process_csv_file(filename)

Se encontraron 30 archivos CSV.

[1/30]
Procesando archivo: L220H12G16_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 12, 'G': 16}
  → Despiking en columna: V1/X/E(cm/s)
  → Despiking en columna: V2/Y/N(cm/s)
  → Despiking en columna: V3/Z/U(cm/s)
  ✅ Guardado como: ./clean_data/L220H12G16_Probe_1_(A837F)._clean.csv

[2/30]
Procesando archivo: L220H15G16_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 15, 'G': 16}
  → Despiking en columna: V1/X/E(cm/s)
  → Despiking en columna: V2/Y/N(cm/s)
  → Despiking en columna: V3/Z/U(cm/s)
  ✅ Guardado como: ./clean_data/L220H15G16_Probe_1_(A837F)._clean.csv

[3/30]
Procesando archivo: L220H18G14_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 18, 'G': 14}
  → Despiking en columna: V1/X/E(cm/s)
  → Despiking en columna: V2/Y/N(cm/s)
  → Despiking en columna: V3/Z/U(cm/s)
  ✅ Guardado como: ./clean_data/L220H18G14_Probe_1_(A837F)._clean.csv

[4/30]
Procesando archivo: L220H22G14_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 22, 'G': 14}
  → Despiki